Original dataset:

https://earth-info.nga.mil/php/download.php?file=egm-08spherical

In [25]:
# Load the string representations of the C/S
# coefficients up to the maximum degree into
# a list of tuples.
max_degree=80
CS_list = []
with open("EGM2008_to2190_TideFree", "r") as f:
    for line in f:
        fields = line.split()
        cur_deg = int(fields[0])
        if cur_deg > max_degree:
            break
        # NOTE: replace 'D' with 'e' for parsing
        # of doubles in C++.
        cur_C = fields[2].replace('D','e')
        cur_S = fields[3].replace('D','e')
        CS_list.append((cur_C, cur_S))

# Explanation: for a degree n, there are n+1 orders.
# Hence, at degree n we expect a total of (n+1)*(n+2)/2
# coefficients, minus 3 because the first coefficient order
# is 2 and not 0.
assert len(CS_list) == ((max_degree+1)*(max_degree + 2)) // 2 - 3

In [19]:
# Template for the header file output.
HPP_FILE_TMPLT = r"""#ifndef HEYOKA_DETAIL_EGM2008_HPP
#define HEYOKA_DETAIL_EGM2008_HPP

#include <heyoka/config.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

extern const double egm2008_CS[{}][2];

}} // namespace model::detail

HEYOKA_END_NAMESPACE

#endif

"""

# Template for the cpp file output.
CPP_FILE_TMPLT = r"""#include <heyoka/config.hpp>
#include <heyoka/detail/egm2008.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

const double egm2008_CS[{0}][2] = {1};

}} // namespace model::detail

HEYOKA_END_NAMESPACE

"""

In [33]:
# Write the files.
with open('egm2008.hpp', 'w') as f:
    f.write(HPP_FILE_TMPLT.format(len(CS_list)))

with open('egm2008.cpp', 'w') as f:
    f.write(CPP_FILE_TMPLT.format(len(CS_list), f'{{{',\n'.join(f'{{{_[0]},{_[1]}}}' for _ in CS_list)}}}'))